# Numpy Compat

In [1]:
from uarray import *
from uarray.ast import *
import numpy as np
import matchpy
import ast
import typing

In [2]:
@optimize(final_call=lambda e: e)
def some_fn(a, b):
    return np.multiply.outer(a, b)[10]

In [3]:
e = some_fn(ArrayLike(with_dims(Expression(ast.Name("a", ast.Load())), 1)), ArrayLike(with_dims(Expression(ast.Name("b", ast.Load())), 1)))
e

Sequence(Length(Expression(Name(id='b', ctx=Load()))),
         Function(Scalar(Multiply(Content(Call(Content(Expression(Name(id='a', ctx=Load()))),
                                               Value('10'))),
                                  Content(Call(Content(Expression(Name(id='b', ctx=Load()))),
                                               Unbound('', variable_name=i5))))),
                  Unbound('', variable_name=i5)))

In [4]:
replace(e)

Sequence(Length(Expression(Name(id='b', ctx=Load()))),
         Function(Scalar(Multiply(Content(Call(Content(Expression(Name(id='a', ctx=Load()))),
                                               Value('10'))),
                                  Content(Call(Content(Expression(Name(id='b', ctx=Load()))),
                                               Unbound('', variable_name=i5))))),
                  Unbound('', variable_name=i5)))

In [5]:
replace(AssignAsNPArray(replace(e), Value("res"), Value(True)))

(AssignAsNestedList(Sequence(Value('1'),
                            VectorCallable(Length(Expression(Name(id='b', ctx=Load()))))),
                   Value("'id0'")),
 Statement(Assign(targets=[Name(id='res', ctx=Store())], value=Call(func=Attribute(value=Name(id='np', ctx=Load()), attr='array', ctx=Load()), args=[Call(func=Name(id='tuple', ctx=Load()), args=[Name(id='id0', ctx=Load())], keywords=[])], keywords=[]))),
 Call(Call(SubstituteStatements(<function assign_sequence_nparray.<locals>.inner at 0x110c57d90>),
          AssignAsValue(Length(Expression(Name(id='b', ctx=Load()))),
                        Value("'id2'"))),
     Call(Call(SubstituteStatements(<function _assign_multiply.<locals>.inner at 0x110c3bf28>),
               AssignAsValue(Content(Call(Content(Expression(Name(id='a', ctx=Load()))),
                                          Value('10'))),
                             'id4')),
          AssignAsValue(Content(Call(Content(Expression(Name(id='b', ctx=Load()))),
  

In [6]:
%time some_fn.__wrapped__(*args)

NameError: name 'args' is not defined

In [7]:
# import logging

# logging.basicConfig(level=logging.DEBUG)


In [8]:
%time some_fn(*args)

NameError: name 'args' is not defined

## Tensorflow support

Even though we support the NumPy API, that doesn't tie us to using NumPy arrays. Instead let's add support for TensorFlow arrays and use those:

In [9]:
import tensorflow as tf

We have to define a way to take the final reduced expression and return a tensorflow graph.

We can do this by creating an empty TF array, and looping over all possible

In [10]:
class TFTensor(matchpy.Symbol):
    pass

In [11]:
class ToTFTensor(matchpy.Operation):
    name = "ToTFTensor"
    arity = matchpy.Arity(1, False)

In [12]:
class ToTFTensorInner(matchpy.Operation):
    name = "ToTFTensorInner"
    arity = matchpy.Arity(1, False)

In [13]:
register(ToTFTensor(Scalar(w.x)), lambda x: Scalar(ToTFTensor(x)))
register(ToTFTensor(Value.w.x), lambda x: ToTFTensorInner(x))

In [14]:
register(ToTFTensorInner(Value.w.x), lambda x: Content(TFTensor(tf.convert_to_tensor(x.value))))

In [15]:
register(ToTFTensor(Sequence(w.length, w.content)),lambda length, content: ToTFTensorInner(
    Sequence(
        ToTFTensor(length),
        ToGenerateTFTensor(Call(content, Content(GenerateTFTensor(lambda i: i))))
    )
))

In [16]:
class GenerateTFTensor(matchpy.Symbol):
    """
    GenerateTFTensor(lambda i: tf.Tensor(...))
    """
    pass

In [17]:
class ToGenerateTFTensor(matchpy.Operation):
    name = "ToGenerateTFTensor"
    arity = matchpy.Arity(1, False)

In [18]:
register(
    ToTFTensorInner(Sequence(Content(TFTensor.w.length), GenerateTFTensor.w.content)),
    lambda length, content: TFTensor(tf.stack(tf.map_fn(
        content.name,
        tf.range(length.name)
    ))))

In [19]:
register(Scalar(Content(GenerateTFTensor.w.x)), lambda x: x)
register(ToGenerateTFTensor(GenerateTFTensor.w.x), lambda x: x)


In [20]:
expr = uarray.OuterProduct(function(2, Multiply), uarray.Iota(uarray.scalar(100)), uarray.Iota(uarray.scalar(10)))

NameError: name 'uarray' is not defined

In [ ]:
register(ToGenerateTFTensor(Sequence(w.x, )))

In [ ]:
replace(ToTFTensor((replace(expr))))

In [ ]:
g_1 = tf.Graph()

In [ ]:
with g_1.as_default():
    with tf.Session():
        res = uarray.replace(expr).name
        print(res.eval())

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(g_1.as_graph_def())


In [ ]:
import numba

In [ ]:
@numba.njit
def hi():
    return np.empty([1, 2])

In [ ]:
hi()

In [ ]:
class A(matchpy.Symbol):
    pass

In [ ]:
def _a(x):
    yield Valuast.parse("print('hello world')")e(1)
    yield Value(2)
register(A.w.x, _a)

In [ ]:
replace(uarray.core.VectorCallable(A("df"), A("df")))

In [22]:
ast.dump(ast.parse("a.shape"))

"Module(body=[Expr(value=Attribute(value=Name(id='a', ctx=Load()), attr='shape', ctx=Load()))])"

In [ ]:
list(1)

In [ ]:
np.inner(np.arange(10), np.arange(10))

In [ ]:
a = np.empty(tuple())

In [ ]:
a